In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
#importamos los ratings para cada lugar
Y_data = pd.read_csv('ratings_place.csv', sep=",", header = None).values
Y_data.shape

(34, 57)

In [3]:
#importamos la lista de lugares
places_list = pd.read_csv('locations.csv', sep=",", skiprows =[0], usecols=[1], header = None).values
places_list.shape

(34, 1)

In [4]:
#Logica del filtrado colaborativo para el minado de reglas de asociacion

Y = tf.placeholder(tf.float32)

X = tf.Variable(tf.truncated_normal([34, 100], stddev=0.1))
Theta = tf.Variable(tf.truncated_normal([57, 100], stddev=0.1))

##### costos
R = tf.cast(tf.not_equal(Y, 0), tf.float32)

Y_mean = (tf.reduce_sum(Y, axis = 1) / tf.reduce_sum(R, axis = 1))

Y_norm = Y - tf.reshape(Y_mean,[-1,1])

####

cost = 0.5 * tf.reduce_sum(R * (tf.square(tf.matmul(X, tf.transpose(Theta)) - Y_norm)))

train_step = tf.train.AdamOptimizer(1e-3).minimize(cost)

#predicciones
prediction = tf.matmul(X, tf.transpose(Theta)) + tf.reshape(Y_mean,[-1,1])

#creamos la sesion de tf
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

#entrenamiento
for i in range(10000):
    if i % 100 == 0:
        print('step %d, training cost %g' % (i, cost.eval(feed_dict={Y: Y_data})))
    sess.run(train_step, feed_dict={Y: Y_data})

Y_pred = prediction.eval(feed_dict={Y: Y_data})
print('Predicted Y:\n', Y_pred)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
step 0, training cost 571.881
step 100, training cost 176.67
step 200, training cost 21.3566
step 300, training cost 1.36835
step 400, training cost 0.0856943
step 500, training cost 0.00772991
step 600, training cost 0.000913642
step 700, training cost 0.000127933
step 800, training cost 1.95482e-05
step 900, training cost 2.98998e-06
step 1000, training cost 4.31155e-07
step 1100, training cost 5.71507e-08
step 1200, training cost 7.08022e-09
step 1300, training cost 9.18824e-10
step 1400, training cost 1.59096e-10
step 1500, training cost 5.13174e-11
step 1600, training cost 2.79093e-11
step 1700, training cost 1.86653e-11
step 1800, training cost 1.39681e-11
step 1900, training cost 1.0659e-11
step 2000, training cost 1.07713e-11
step 2100, training cost 1.77793e-11
step 2200, training cost 3.63028e-09
step 2300, training cost 6.12426e-12
step 2400, training cost 

In [5]:

recommend = np.argsort(Y_pred, axis=0)


# Muestra las k mejores recomendaciones para el usuario j
k = 10
j = 5

for i in range (k):
    print(places_list[recommend[i,j],0])


Malecon Escenico Murcielago
Museo Centro Cultural Manta
Playa El Murciélago
Monumento A Eloy Alfaro D.
Gran Aki Tarqui
Parque comercial Nuevo Tarqui
Navy`s Bar Discoteca
Museo Municipal Etnográfico Cancebí
La Plaza Beach
Noa Bar Grill


In [6]:
#ahora cargamos los perfiles por usuario en un df

In [7]:
sess.close()

In [8]:
visual = pd.DataFrame(recommend)
visual

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
0,1,1,1,1,1,1,1,1,1,1,...,29,29,29,29,29,29,7,29,29,29
1,4,4,27,15,15,4,3,16,18,2,...,22,7,28,31,7,7,29,22,17,7
2,16,2,17,18,30,2,16,6,21,6,...,31,31,7,22,31,31,17,17,31,22
3,10,11,25,6,25,6,8,18,25,9,...,7,17,17,28,17,17,28,28,7,9
4,14,3,24,32,3,24,23,21,31,25,...,28,22,31,17,28,22,31,31,28,12
5,9,15,5,0,4,23,25,25,24,28,...,23,9,9,7,22,28,0,7,9,31
6,13,16,3,24,16,30,24,24,13,33,...,17,28,33,23,9,9,9,23,22,28
7,3,10,4,28,10,0,4,17,17,31,...,9,23,22,30,33,23,30,30,30,17
8,15,26,15,3,2,22,10,5,5,5,...,32,5,12,9,30,5,5,9,0,23
9,2,6,10,4,8,31,15,29,3,29,...,33,12,30,12,23,12,14,12,5,5


In [9]:
np.save('train_matrix.npy', recommend)
rec_tensor = tf.stack(recommend)

In [15]:
import tensorflow.lite as lite
from tensorflow.lite.toco.python import toco_from_protos

users = pd.read_csv('user.csv', sep=",", header = None, skiprows=[0], usecols=[1,2,3]).values

inp = tf.placeholder(name="inp", dtype=tf.float32, shape=(1, 3))

tuser = tf.Variable(users, dtype=tf.float32)

tplace = tf.Variable(recommend, dtype=tf.float32)

tplace = tf.transpose(tplace)

normed_inp = tf.nn.l2_normalize(inp, axis=1)
normed_users = tf.nn.l2_normalize(tuser, axis=1)

cosine_similarity = tf.matmul(normed_users, tf.transpose(normed_inp, [1, 0]))

closest = tf.argmax(cosine_similarity, 0)

places = tf.gather(tplace, closest)

init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init_op)
    output = sess.run(places, feed_dict={inp: [[20,1,1]]})
    print(output)
    
    
    converter = lite.TFLiteConverter.from_session(sess, [inp], [places])
    tflite_model = converter.convert()
    open("recsys.tflite", "wb").write(tflite_model)


[[ 1. 15. 30. 25.  3.  4. 16. 10.  2.  8. 26. 11. 21. 27. 32. 28. 33. 12.
  24. 13. 20. 14. 18. 19.  6.  0.  7.  9. 22. 17. 31.  5. 29. 23.]]
Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.


In [16]:
#import tensorflow as tf

interpreter = tf.lite.Interpreter(model_path="recsys.tflite")
interpreter.allocate_tensors()

# Print input shape and type
print(interpreter.get_input_details()[0]['shape'])  # Example: [1 224 224 3]
print(interpreter.get_input_details()[0]['dtype'])  # Example: <class 'numpy.float32'>

# Print output shape and type
print(interpreter.get_output_details()[0]['shape'])  # Example: [1 1000]
print(interpreter.get_output_details()[0]['dtype'])  # Example: <class 'numpy.float32'>

[1 3]
<class 'numpy.float32'>
[ 1  1 34]
<class 'numpy.float32'>
